# 빈도분석 + 키워드 추출
## 불용어 리스트인 stopwords.txt 다운로드 필수
## 키워드 추출 결과가 저장된 변수: result (리스트 형태)

In [11]:
# 모듈 import
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from collections import Counter
from pykospacing import Spacing #pip install git+https://github.com/haven-jeon/PyKoSpacing.git 
import time

In [28]:
# 빈도분석 + 키워드 추출 함수

def load_area():
    df = pd.read_csv('경기도 양평군.csv', encoding='utf-8')
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df_body = df.iloc[:, 1:2]
    body = df_body['블로그내용']
    return body

def load_stopwords():
    with open('stopwords.txt', 'r') as f:
        list_file = f.readlines()
    return list_file[0].split(",")

# 정규 표현식을 통해 한글 단어만 남기기 (이모티콘, 초성, 영어 제거)
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text)
    return result

def make_wordlist(body,stopwords):
    #정규표현식 적용
    print("데이터 정제 중....")
    words = body.apply(lambda x:extract_word(x))
    spacing = Spacing()
    words = words.apply(lambda x:spacing(x))
    
    #형태소 추출
    print("형태소 추출 중....")
    okt = Okt()
    words = " ".join(words.tolist())
    words = okt.morphs(words,stem=True)
    
    #한글자 제거
    print("한글자 제거 중....")
    words = [x for x in words if len(x)>1]
    
    #불용어 제거
    print("불용어 제거 중....")
    words = [x for x in words if x not in stopwords]
    
    #최소횟수 미만 제거
    print("키워드 빈도 수 기준 상위 20개 리스트 생성 중....")
    time.sleep(1)
    extraction = Counter(words).most_common()[0:20]
    result = [x[0] for x in extraction]
    return result


def run():
    body = load_area()
    print(make_wordlist(body, load_stopwords()))

In [29]:
run()

데이터 정제 중....
형태소 추출 중....
한글자 제거 중....
불용어 제거 중....
키워드 빈도 수 기준 상위 20개 리스트 생성 중....
['양평', '좋다', '여행', '먹다', '경기도', '지도', '양평군', '카페', '보기', '많다', '맛있다', '펜션', '자다', '거리', '영상', '사진', '허브', '복사', '도움말', '두물머리']
